# Text Encoding

Trong quá trình xử lý dữ liệu thực tế, rất nhiều thuộc tính (cột) xuất hiện dưới dạng văn bản như tên danh mục, trạng thái, mô tả, bình luận,… Tuy nhiên, các mô hình Machine Learning không thể làm việc trực tiếp với dữ liệu dạng text. Vì vậy, bước mã hóa dữ liệu (Text Encoding) là một phần quan trọng trong giai đoạn tiền xử lý dữ liệu (data preprocessing).

Notebook này trình bày các kỹ thuật phổ biến để chuyển đổi văn bản thành dạng số, giúp mô hình có thể hiểu và học từ dữ liệu:

1. One-Hot Encoding – biến mỗi giá trị phân loại thành một vector nhị phân
2. Target Encoding – mã hoá theo mục tiêu

# 0. Nguồn dữ liệu

In [1]:
import pandas as pd
import numpy as np
import unidecode
from sklearn.model_selection import KFold

df = pd.read_csv('../data/interim/combined_interim.csv')
THRES_SHOLD = 0.005

# 1. One-hot field
Một số trường dữ liệu có loại dữ liệu thuộc một tập hữu hạn trong khoảng dưới 30 loại (có thể chọn đến 50) sẽ được encoding theo one-hot

## 1.1. Mapping

Ít dữ liệu sử dụng mapping trực tiếp là được

In [2]:
def normalize_categorical(x, mapping=None, keywords_map=None):
    """
    x: giá trị cần chuẩn hóa
    mapping: dict {giá trị chuẩn hóa: value mới}, khớp exact
    keywords_map: dict {từ khóa: value mới}, nếu từ khóa xuất hiện trong x -> map
    """
    if pd.isna(x):
        return np.nan

    val = str(x).lower().strip()

    # 1. khớp exact mapping
    if mapping and val in mapping:
        return mapping[val]

    # 2. khớp theo từ khóa
    if keywords_map:
        for kw, kw_val in keywords_map.items():
            if kw in val:
                return kw_val

    # 3. fallback
    return val

### 1.1.1 Status

In [3]:
status_mapping = {
    'đã sử dụng': 'used',
    'xe đã dùng': 'used',
    'xe cu': 'used',
    'cũ': 'used',
    'xe mới': 'new',
    'mới': 'new'
}

df['status'] = df['status'].apply(lambda x: normalize_categorical(x, mapping=status_mapping))
print(df['status'].value_counts(dropna=False))

status
used    12437
new      2491
Name: count, dtype: int64


### 1.1.2. Fuel

In [4]:
fuel_mapping = {
    'xăng': 'gasoline',
    'dầu': 'diesel',
    'dầu diesel': 'diesel',
    'điện': 'electric',
    'hybrid': 'hybrid'
}

df['fuel_type'] = df['fuel_type'].apply(lambda x: normalize_categorical(x, mapping=fuel_mapping))
print(df['fuel_type'].value_counts(dropna=False))

fuel_type
gasoline          11402
diesel             2209
electric            841
hybrid              443
động cơ hybrid       33
Name: count, dtype: int64


### 1.1.3. Gear

In [5]:
gear_mapping = {
    'số sàn': 'mt',
    'manual': 'mt',
    'số tự động': 'at',
    'tự động': 'at',
    'automatic': 'at',
    'auto': 'at'
}

# từ khóa phụ, nếu muốn match chuỗi phức tạp
keywords_map = {
    'sàn': 'mt',
    'manual': 'mt',
    'tự động': 'at',
    'auto': 'at',
    'at': 'at'
}

df['gear'] = df['gear'].apply(lambda x: normalize_categorical(x, mapping=gear_mapping, keywords_map=keywords_map))
print(df['gear'].value_counts(dropna=False))

gear
at     7099
NaN    5851
mt     1960
5        17
4         1
Name: count, dtype: int64


### 1.1.4. Style

In [6]:
style_mapping = {
    'suv / cross over': 'suv',
    'suv': 'suv',
    'sedan': 'sedan',
    'hatchback': 'hatchback',
    'minivan (mpv)': 'minivan',
    'van/minivan': 'minivan',
    'van': 'minivan',
    'bán tải / pickup': 'pickup',
    'pick-up (bán tải)': 'pickup',
    'coupe': 'coupe',
    'coupe (2 cửa)': 'coupe',
    'convertible/cabriolet': 'convertible',
    'kiểu dáng khác': 'other',
}

df['style'] = df['style'].apply(lambda x: normalize_categorical(x, mapping=style_mapping))

## 1.2. Gom Nhóm

Nhiều cột hơn cần gom các mẫu rất nhỏ tránh làm sai mô hình

In [7]:
def normalize_and_group(df, col, threshold_ratio=0.005, other_label='other'):
    """
    Chuẩn hóa tên và gom nhóm các giá trị ít xuất hiện thành 'other'.

    df: DataFrame
    col: tên cột cần xử lý
    threshold_ratio: tỷ lệ tối thiểu (ví dụ 0.005 = 0.5%)
    other_label: nhãn cho các giá trị hiếm
    """
    # 1. Chuẩn hóa tên: lowercase, không dấu, replace space bằng _
    df[col] = df[col].apply(lambda x: unidecode.unidecode(str(x)).lower().replace(' ', '_'))

    # 2. Đếm tần suất
    counts = df[col].value_counts()

    # 3. Xác định threshold
    total_samples = len(df)
    threshold = threshold_ratio * total_samples

    # 4. Gom nhóm các giá trị ít xuất hiện
    df[col] = df[col].apply(lambda x: x if counts[x] >= threshold else other_label)

    # 5. Trả về DataFrame đã xử lý
    return df

### 1.2.1. Origin

In [8]:
df = normalize_and_group(df, 'origin', threshold_ratio=THRES_SHOLD)

# Kiểm tra kết quả
print(df['origin'].value_counts())

origin
trong_nuoc       5159
nhap_khau        4841
viet_nam         1989
dang_cap_nhat    1095
thai_lan          426
han_quoc          375
nhat_ban          327
nuoc_khac         268
duc               148
my                147
trung_quoc         83
other              70
Name: count, dtype: int64


### 1.2.2. Location

In [9]:
df = normalize_and_group(df, 'location', threshold_ratio=THRES_SHOLD)

# Kiểm tra kết quả
print(df['location'].value_counts())

location
ha_noi               7290
hcm                  2040
ho_chi_minh          1819
other                1109
binh_duong            415
da_nang               342
hai_phong             313
dong_nai              288
nan                   164
can_tho               158
dak_lak               156
lam_dong              114
unknown               107
phu_tho               104
vinh_phuc             103
ba_ria_-_vung_tau      93
bac_ninh               82
gia_lai                79
nghe_an                76
thanh_hoa              76
Name: count, dtype: int64


### 1.2.3. Brand

In [10]:
df = normalize_and_group(df, 'brand', threshold_ratio=THRES_SHOLD)

# Kiểm tra kết quả
print(df['brand'].value_counts())

brand
toyota           2422
ford             1651
mercedes_benz    1482
hyundai          1323
kia              1194
vinfast          1082
mazda             732
mitsubishi        731
lexus             577
honda             564
bmw               497
other             457
chevrolet         305
mg                233
suzuki            214
landrover         186
porsche           185
nissan            177
peugeot           171
volkswagen        164
audi              157
daewoo            147
volvo             108
isuzu              89
hang_khac          80
Name: count, dtype: int64


## 2. Target Encoding

Đọ đa dạng của series rất lớn hơn 1300 loại không thể dùng One-hot, ta sử dụng target encoding

### 2.1. K-fold

In [11]:
def kfold_target_encode(train_series, target, test_series=None, n_splits=5, min_samples_leaf=1, smoothing=1, random_state=None):
    """
    train_series: categorical series của train
    target: numeric target series của train
    test_series: categorical series của test (nếu có)
    n_splits: số fold
    min_samples_leaf: minimum samples cho smoothing
    smoothing: hệ số smoothing
    """
    # Global mean
    global_mean = target.mean()

    # KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    train_encoded = pd.Series(index=train_series.index, dtype=float)

    for train_idx, val_idx in kf.split(train_series):
        # Split
        tr, val = train_series.iloc[train_idx], train_series.iloc[val_idx]
        tr_target = target.iloc[train_idx]

        # Compute category mean & count
        averages = tr_target.groupby(tr).agg(['mean', 'count'])

        # Smoothing
        smoothing_factor = 1 / (1 + np.exp(-(averages['count'] - min_samples_leaf) / smoothing))
        averages['te'] = global_mean * (1 - smoothing_factor) + averages['mean'] * smoothing_factor

        # Map to validation fold
        train_encoded.iloc[val_idx] = val.map(averages['te'])

    # Encode test set if given
    if test_series is not None:
        # Compute category mean on full train
        averages_full = target.groupby(train_series).agg(['mean', 'count'])
        smoothing_factor = 1 / (1 + np.exp(-(averages_full['count'] - min_samples_leaf) / smoothing))
        averages_full['te'] = global_mean * (1 - smoothing_factor) + averages_full['mean'] * smoothing_factor

        test_encoded = test_series.map(averages_full['te']).fillna(global_mean)
        return train_encoded, test_encoded

    return train_encoded

### 2.1. Series

Cột model/series có quá nhiều giá trị duy nhất, ta sử dụng target encoding để chuyển thành giá trị số dựa trên mối quan hệ với giá (price)

In [12]:
# Chuẩn hóa tên series
df['series'] = df['series'].apply(lambda x: unidecode.unidecode(str(x)).lower().replace(' ', '_') if pd.notna(x) else x)

# Kiểm tra số lượng giá trị duy nhất
print(f"Số lượng series duy nhất: {df['series'].nunique()}")
print(f"Top 10 series phổ biến:\n{df['series'].value_counts().head(10)}")

# Áp dụng target encoding cho cột series
# Sử dụng KFold để tránh data leakage
df['series_encoded'] = kfold_target_encode(
    train_series=df['series'],
    target=df['price'],
    n_splits=5,
    min_samples_leaf=1,
    smoothing=1,
    random_state=42
)

# Xử lý NaN: fill bằng global mean của price
df['series_encoded'] = df['series_encoded'].fillna(df['price'].mean())

print(f"\n✓ Đã áp dụng Target Encoding cho cột 'series'")
print(f"Giá trị series_encoded:\n{df['series_encoded'].describe()}")

# Giữ lại cột series gốc để tham khảo, sẽ xóa trước khi train model
print(f"\nDữ liệu mẫu:")
print(df[['series', 'series_encoded', 'price']].head(10))

Số lượng series duy nhất: 1342
Top 10 series phổ biến:
series
glc_00        321
vf_plus       317
innova        301
camry         233
vios          232
fortuner      218
dong_khac     188
xpander       181
ranger_xls    171
morning       162
Name: count, dtype: int64

✓ Đã áp dụng Target Encoding cho cột 'series'
Giá trị series_encoded:
count    1.492800e+04
mean     1.046884e+09
std      1.308730e+09
min      3.610082e+07
25%      4.209245e+08
50%      6.314865e+08
75%      1.082838e+09
max      2.304142e+10
Name: series_encoded, dtype: float64

Dữ liệu mẫu:
        series  series_encoded       price
0           x5    2.520583e+09   320000000
1      e_class    7.409630e+08   108000000
2      transit    4.331567e+08   365000000
3        cruze    2.154721e+08   339000000
4        camry    6.242234e+08   380000000
5  range_rover    1.586419e+09  2450000000
6         zace    1.191042e+08   170000000
7           rx    1.024430e+09   740000000
8         dmax    3.130203e+08   450000000
9   

# 3. Lưu thay đổi

In [13]:
# === Target Encoding - Drop original series column ===
# Xóa cột series gốc vì đã có series_encoded
if 'series' in df.columns:
    df = df.drop(columns=['series'])
    print("✓ Đã xóa cột 'series' gốc, giữ lại 'series_encoded'")

# === One-Hot Encoding ===
onehot_cols = ['status', 'fuel_type', 'origin', 'style', 'brand', 'gear', 'location']

df = pd.get_dummies(df, columns=onehot_cols, prefix=onehot_cols, dtype=int)

# === Chuẩn hóa kiểu dữ liệu số ===
numeric_cols = ['price', 'odometer', 'seats', 'year']

for col in numeric_cols:
    if col not in df.columns:
        continue

    # Price luôn int64
    if col == 'price':
        df[col] = df[col].astype('int64')

    # Các cột còn lại: nếu có NaN -> dùng Int64; nếu không -> int64
    else:
        df[col] = df[col].astype('Int64') if df[col].isna().any() else df[col].astype('int64')

# series_encoded giữ nguyên kiểu float64 vì là target encoding
if 'series_encoded' in df.columns:
    df['series_encoded'] = df['series_encoded'].astype('float64')

# === Xuất CSV ===
csv_file = '../data/interim/encoding_interim.csv'
df.to_csv(csv_file, index=False, encoding='utf-8-sig')

# === Thông tin ===
print("\n✓ Đã áp dụng One-Hot Encoding")
print(f"Số cột sau One-Hot Encoding: {df.shape[1]}")
print("\nCác cột one-hot tạo ra:")
print([c for c in df.columns if any(c.startswith(prefix + "_") for prefix in onehot_cols)])
print("\nDữ liệu sau encoding:")
print(df.head())

✓ Đã xóa cột 'series' gốc, giữ lại 'series_encoded'

✓ Đã áp dụng One-Hot Encoding
Số cột sau One-Hot Encoding: 86

Các cột one-hot tạo ra:
['status_new', 'status_used', 'fuel_type_diesel', 'fuel_type_electric', 'fuel_type_gasoline', 'fuel_type_hybrid', 'fuel_type_động cơ hybrid', 'origin_dang_cap_nhat', 'origin_duc', 'origin_han_quoc', 'origin_my', 'origin_nhap_khau', 'origin_nhat_ban', 'origin_nuoc_khac', 'origin_other', 'origin_thai_lan', 'origin_trong_nuoc', 'origin_trung_quoc', 'origin_viet_nam', 'style_convertible', 'style_coupe', 'style_crossover', 'style_hatchback', 'style_minivan', 'style_mui trần', 'style_other', 'style_pickup', 'style_sedan', 'style_suv', 'style_truck', 'style_wagon', 'brand_audi', 'brand_bmw', 'brand_chevrolet', 'brand_daewoo', 'brand_ford', 'brand_hang_khac', 'brand_honda', 'brand_hyundai', 'brand_isuzu', 'brand_kia', 'brand_landrover', 'brand_lexus', 'brand_mazda', 'brand_mercedes_benz', 'brand_mg', 'brand_mitsubishi', 'brand_nissan', 'brand_other', 'bran